# Homework 4 - pandas (Leah Prescott)  
### due 3/14/2023 

Use this notebook and prompts to complete the homework. Throughout there will be hints and some code provided  

### Things you will need:  
- Install os, NumPy, pandas  
- states_covid.csv  
- Bloom_etal_2018_Reduced_Dataset.csv  
- logfiles.tgz (or some other multiple file dataset)  

*NOTE*: Make sure your PATH is correct  

**import packages & check required datasets**   

In [1]:
import os
import numpy as np
import pandas as pd

In [3]:
PATH = '/Users/leah/Desktop/Python_Class/F22_BIOL792_coursepage/week11_python6' 

In [4]:
assert os.path.exists(os.path.join(PATH,'states_covid.csv')), 'states_covid.csv does not exist' 

In [5]:
assert os.path.exists(os.path.join(PATH,'Bloom_etal_2018_Reduced_Dataset.csv')), 'Bloom_etal_2018_Reduced_Dataset.csv does not exist'

## Task 1 - DataFrame manipulation  

Using **states_covid.csv**, we are going to read the data in as a DataFrame to manipulate, subset, and filter in various ways.   

### Task 1.1 

Read in states_covid.csv with date as a "date" dtype, and only columns consisting of the hospitalization (4 col), ICU (2 col), and Ventilators (2 col)  

In [64]:
state_covid_df = pd.read_csv('states_covid.csv') #read in csv

state_covid_sub_df = pd.read_csv('states_covid.csv', 
usecols=['date', 'state','hospitalized', 'hospitalizedCumulative','hospitalizedCurrently', 'hospitalizedIncrease','inIcuCumulative','inIcuCurrently','onVentilatorCumulative', 'onVentilatorCurrently'], 
parse_dates=['date'], infer_datetime_format=True)

### Task 1.2 

For each of the following catergories: *currently* hospitalized, *currently* in the ICU, and *currently* on ventilation...  
Find the 5 states with the greatest numbers in each catergory and list them in decreasing order.      
*hint*: sort_values, unique  

In [82]:
statecovid2 = state_covid_sub_df.groupby('state',as_index=True).agg(['sum'])
statecovid2.columns = statecovid2.columns.map('_'.join)

hospital_df2 = statecovid2.sort_values('hospitalizedCurrently_sum',ascending=False)
#print(hospital_df2[0:5])
print(hospital_df2.head(5))

icu_df = statecovid2.sort_values('inIcuCurrently_sum',ascending=False)
#print(icu_df[0:5])
print(icu_df.head(5))


ventilation_df = statecovid2.sort_values('onVentilatorCurrently_sum',ascending=False)
#print(ventilation_df[0:5])
print(ventilation_df.head(5))


       hospitalized_sum  hospitalizedCumulative_sum  \
state                                                 
CA                  0.0                         0.0   
TX                  0.0                         0.0   
NY           28486669.0                  28486669.0   
FL           12473211.0                  12473211.0   
IL                  0.0                         0.0   

       hospitalizedCurrently_sum  hospitalizedIncrease_sum  \
state                                                        
CA                     2588299.0                         0   
TX                     2062892.0                         0   
NY                     1550375.0                     89995   
FL                     1106003.0                     79410   
IL                      957999.0                         0   

       inIcuCumulative_sum  inIcuCurrently_sum  onVentilatorCumulative_sum  \
state                                                                        
CA                     

/var/folders/2h/2bnmh4414194h1khd5r2g7v80000gn/T/ipykernel_92002/1854193038.py:1: FutureWarning: ['date'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  statecovid2 = state_covid_sub_df.groupby('state',as_index=True).agg(['sum'])


### Task 1.3 

Find the date in which each state crossed 1000 cumulative hospitilized covid patients

In [21]:
querystate = state_covid_sub_df.query("hospitalizedCumulative >= 1000")[['date','state']]

print(querystate.groupby('state').agg('min)'))

,date,state
0,2021-02-23,AK
1,2021-02-23,AL
2,2021-02-23,AR
4,2021-02-23,AZ
6,2021-02-23,CO
...,...,...
18797,2020-03-25,NY
18853,2020-03-24,NY
18909,2020-03-23,NY
18965,2020-03-22,NY


## Task 2 - DataFrame summarizing  

Using **Bloom_etal_2018_Reduced_Dataset.csv**, we are going to do more dataframe manipulation and subsetting and summarizing    

### Task 2.1 

Read in Bloom_etal_2018_Reduced_Dataset.csv and create two new columns ('genus','species') that consists of the column *taxa* split at the underscore. Print out the head of this new dataframe and the number of unique genera**     

*hint:* pd.str.split(,expand=True)  

for example:  

| taxa | genus | species   
| :------ | :-- | :---   
| Alosa_alabamae | Alosa | alabamae  

In [49]:
Bloom = pd.read_csv('Bloom_etal_2018_Reduced_Dataset.csv') #read in csv
Bloom.columns
GenusSpecies = Bloom['taxa'].str.split('_', expand=True)
GenusSpecies
Bloom[['genus', 'species']] = GenusSpecies[[0,1]]

### Task 2.2 

Create a new dataframe with the mean *logbodysize* and *trophicposition* of each genera. Sort this data frame by the largest body size. Print the head of this dataframe.

In [51]:
bloom_reg_df = Bloom.groupby('genus',as_index=True).agg(['mean'])
bloom_reg_df
bloom_reg_df.columns = bloom_reg_df.columns.map('_'.join)
bloom_reg_df.sort_values('logbodysize_mean',ascending=False,inplace=True)
bloom_reg_df.head()

/var/folders/2h/2bnmh4414194h1khd5r2g7v80000gn/T/ipykernel_92002/4129539936.py:1: FutureWarning: ['taxa', 'Reg', 'species'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  bloom_reg_df = Bloom.groupby('genus',as_index=True).agg(['mean'])


,logbodysize_mean,trophic_position_mean
genus,,
Tenualosa,1.778151,0.462398
Alosa,1.739062,0.540815
Coilia,1.544068,0.477121
Ethmalosa,1.544068,0.397940
Potamalosa,1.505150,0.518514


Which genera is the smallest and largest?

In [60]:
print(bloom_reg_df.iloc[[0,bloom_reg_df.shape[0]-1]])

                logbodysize_mean  trophic_position_mean
genus                                                  
Tenualosa               1.778151               0.462398
Amazonsprattus          0.301030               0.531479


What is the trophic position of the smallest and largest?  

In [ ]:
print(bloom_reg_df.iloc[[0,bloom_reg_df.shape[0]-1]])

## Task 3 - Read in muliple files to a dictionary and make a DataFrame - **OPTIONAL/BONUS**  

### This is not something you are expected to do in this course, but just here to give you an idea of the things that you COULD do. Answers will be posted after due date.  


Using **logfiles**: we are going to do read in each file, get some data, append it to a dictionary to later make into a dataframe.     

**note:** *make sure to unzip logfiles*

In [20]:
cd $PATH

/Users/trevorfaske/Desktop/Classes/DataScience/pandas/Spring22


In [21]:
!tar -xzf logfiles.tgz #unzip logfiles

In [22]:
log_dir = os.path.join(PATH,'logfiles')
assert os.path.exists(log_dir), 'log_dir does not exist'

First step is to find the necessary files. The number of files in the log files is 36, make sure you have that many as well  

In [23]:
!ls -l logfiles/*txt | wc -l 

      36


In [24]:
logfiles = !find $log_dir -name '*txt' #unix command to find files in log_dir directory
logfiles = [os.path.abspath(x) for x in logfiles] #this finds the full path to the file
print(logfiles[1])

/Users/trevorfaske/g/Classes/DataScience/pandas/Spring22/logfiles/1901302121_H8_S_14.txt.txt


In [25]:
assert(len(logfiles)==36), 'Do not have correct number of logfiles'

### Getting a little tricky here  

Read in each of the logfiles, for each file extract:  
- minimum temperature  
- maximum temperature  
- date of minimum temp   
- date of maximum temp   
- mean temp for each file.   

This data should all be appended for a dictionary within a for loop:    
Key should be the file name without the path or .txt extension  
Values should be (minTemp,maxTemp,minDate,maxDate,meanTemp)

I recommend making this work for one file first, then putting the rest in a for loop to do the rest.  

Below is an example of how to read in one file

*hint:* do not read date in as date object

In [41]:
### set up data frame as you read in each file
infile = pd.read_csv(logfiles[0],sep='\t',engine='python')
infile.columns = ['Index','Date','Time','Temp','Type']
infile = infile[['Date','Time','Temp']]
infile.head()

,Date,Time,Temp
0,9/16/2013,8:00:00 AM,47.9
1,9/16/2013,8:35:00 AM,48.2
2,9/16/2013,9:10:00 AM,48.7
3,9/16/2013,9:45:00 AM,49.4
4,9/16/2013,10:20:00 AM,50.3


Do everything in steps, make sure it works. Calculate summaries with this one infile:

In [26]:
minTemp =   
maxTemp =  
minDate = infile['Date'][infile['Temp'] == infile['Temp'].min()].unique()[0] #use this for minDate 
maxDate = infile['Date'][infile['Temp'] == infile['Temp'].max()].unique()[0] #use this for maxDate 
meanTemp = 

SyntaxError: invalid syntax (<ipython-input-26-d888355c2242>, line 1)

To get you started, I suggest writing some dummy code in plain words to help outline your for loop:  

In [ ]:
logfiles_dict = {}  
for f in logfiles:  
    #do something   
    #do not read date in as date object  
    #do more something   
    #do other stuff  
    #make print statements EVERYWHERE  
    #append to dict  
    #blahbahblah

Then do the real code below here. You don't need to turnin your thoughts. Just put it in there as a help reminder. Most people all still do this, no matter how advanced they are

**Once you have created a DataFrame with all the logfiles, print the head and save it to an outfile using pd.to_csv() as logfiles_df.csv** 

#### Here is an example of the final product

In [32]:
logfiles_df.head()

,minTemp,maxTemp,minDate,maxDate,meanTemp
1901302235_H6_S_14,21.3,65.8,12/6/2013,7/13/2014,35.726191
1901302121_H8_S_14,17.9,73.4,10/28/2013,7/13/2014,34.698866
1901302217_H18_D_14,21.0,82.4,10/28/2013,7/26/2014,33.334564
1901302110_H13_S_14,22.8,60.9,10/4/2013,7/13/2014,32.240032
1901302146_H14_D_14,24.6,66.1,1/21/2014,7/13/2014,34.242075
